# Repetition code Bell state experiment

## Setup

In [1]:
import functools
import itertools

from typing import List, Dict

import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif","font.size": 12})
import cirq
import qiskit
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

from encoded.repetition_code import logical_H, logical_CNOT
from encoded.LST import int_to_bin_list

In [2]:
import datetime


time_key = datetime.datetime.now().strftime("%m_%d_%Y_%H:%M:%S")  # For saving results.

## Set experiment parameters

In [3]:
# IBM service, computer, and qubits to use.
service = qiskit_ibm_runtime.QiskitRuntimeService()  # This assumes a saved account.
computer = service.backend("ibm_kyiv")
sampler = Sampler(computer)

# See calibration data at https://quantum.ibm.com/services/resources to select good qubits.
layout = {
    1: [3, 4],
    3: [3, 4, 5, 15, 21, 22],
    5: [3, 4, 5, 6, 15, 20, 21, 22, 23, 24],
    7: [0, 1, 2, 3, 4, 5, 6, 14, 15, 18, 19, 20, 21, 22],
}

In [21]:
nvals = [3, 5, 7]      # Numbers of physical qubits (distances of repetition code) to use.

depth: int = 0         # How many times to repeat the circuit U (U^dag U) ^ depth where U is the logical Bell state preparation circuit.
nshots: int = 20_000   # Number of samples.
k = 2                  # Number of logical qubits.

### Helper functions

In [22]:
def generate_stabilizer_elements(generators: List[cirq.PauliString]) -> List[cirq.PauliString]:
    elements = []
    for string in itertools.chain.from_iterable(itertools.combinations(generators, r) for r in range(len(generators) + 1)):
        elements.append(
            functools.reduce(lambda a, b: a * b, string, cirq.PauliString())
        )
    return elements


def compute_expectation(
    pauli: cirq.PauliString,
    counts: Dict[str, int],
) -> float:
    if pauli is cirq.PauliString():
        return 1.0

    expectation = 0.0

    indices = [q.x for q in pauli.qubits]
    for key, value in counts.items():
        key = list(map(int, list(key)))
        expectation += (-1) ** sum([key[i] for i in indices]) * value

    return expectation / sum(counts.values())

def int_to_bin_str(x, length):
    return functools.reduce(lambda a, b: a + b, [str(b) for b in int_to_bin_list(x, length)])


def logical_observable(
    paulistring: str,
    n: int
) -> cirq.PauliString:
    q = cirq.LineQubit.range(n*len(paulistring))
    string = cirq.PauliString()
    for i, p in enumerate(paulistring):
        if p == "X":
            string *= functools.reduce(lambda a, b: a * b, [cirq.X.on(q[j]) for j in range(n*i, n*(i+1))])
        elif p == "Z":
            string *= cirq.Z.on(q[n*i])
    return string

## Run the unmitigated / unencoded experiment as a baseline

In [ ]:
qreg = cirq.LineQubit.range(k)
circuit = cirq.Circuit(
    cirq.H.on(qreg[0]),
    cirq.CNOT.on(*qreg),
)
circuit = qiskit.QuantumCircuit.from_qasm_str(circuit.to_qasm())
circuit.draw()

In [ ]:
# Compile to device.
compiled_raw = qiskit.transpile(
    circuit, 
    backend=computer,
    initial_layout=layout[1],  # Hardcode n = 1 (i.e., no encoding) to get layout.
    routing_method="sabre",
    # scheduling_method="asap",
    optimization_level=3,
)

# Optionally repeat the circuit U -> U (U^\dag U)^depth.
base = compiled_raw.copy()
for _ in range(depth):
    compiled_raw = compiled_raw.compose(base.inverse())
    compiled_raw = compiled_raw.compose(base)
compiled_raw.measure_active()

# Compile one more time with optimization_level=0 (some inverse gates may not be supported).
compiled_raw = qiskit.transpile(compiled_raw, backend=computer, optimization_level=0)

compiled_raw.draw(fold=-1, idle_wires=False)

In [32]:
"""Run the job."""
job = sampler.run(
    [compiled_raw],
    shots=nshots,
)
# job = service.job(job_id="cshov4tfltjvipt17ch0")

In [ ]:
counts = [result.data.measure.get_counts() for result in job.result()][0]
counts

In [ ]:
raw_value = compute_expectation(cirq.Z.on(qreg[0]) * cirq.Z.on(qreg[1]), counts)
raw_value

## Run the mitigated experiment

In [39]:
to_run = []
all_elements = []
all_observables = []
all_paulis = []
for n in nvals:
    # Build the stabilizer group and elements.
    qubits = cirq.LineQubit.range(n * k)
    generators = []
    for i in range(k):
        generators += [cirq.Z.on(qubits[i*n + j]) * cirq.Z.on(qubits[i*n + j + 1]) for j in range(n - 1)]

    elements = generate_stabilizer_elements(generators)  # TODO: Do this in blocks (see Fig. 1 of LST paper).
    all_elements.append(elements)

    # Build the observable and circuit.
    observable = logical_observable("ZZ", n)
    all_observables.append(observable)
    paulis = [observable * p for p in elements]
    all_paulis.append(paulis)

    circuit = cirq.Circuit()
    logical_H(circuit, qubits, n, qi=0)
    logical_CNOT(circuit, qubits, n, 0, 1)
    circuit = circuit[n - 1:]  # Simplify logical H at start of circuit.
    
    # Convert to Qiskit.
    circuit = qiskit.QuantumCircuit.from_qasm_str(circuit.to_qasm())

    # Compile to device.
    compiled = qiskit.transpile(
        circuit, 
        backend=computer,
        initial_layout=layout[n],
        routing_method="sabre",
        # scheduling_method="asap",
        optimization_level=3,
    )

    # Optionally repeat the circuit U -> U (U^\dag U)^depth.
    base = compiled.copy()
    for _ in range(depth):
        compiled = compiled.compose(base.inverse())
        compiled = compiled.compose(base)
    compiled.measure_active()
    
    # Compile one more time with optimization_level=0 (some inverse gates may not be supported).
    compiled = qiskit.transpile(compiled, backend=computer, optimization_level=0)

    to_run.append(compiled)

In [ ]:
to_run[2].draw(fold=-1, idle_wires=False)

In [41]:
"""Run the batch."""
job_batch = sampler.run(
    to_run,
    shots=nshots,
)
# job_batch = service.job(job_id="cshov7ej0jilb25h7hgg")

In [ ]:
job_batch.status()

In [47]:
all_counts = [result.data.measure.get_counts() for result in job_batch.result()]

In [ ]:
all_counts

In [ ]:
# Run the post-processing.
lst_values = []
for i in range(len(all_counts)):
    counts = all_counts[i]
    elements = all_elements[i]
    observable = all_observables[i]
    paulis = all_paulis[i]

    # Compute the error-mitigated estimate.
    numerator = sum(
        compute_expectation(p, counts) for p in paulis
    ) / len(paulis)
    denominator = sum(
        compute_expectation(e, counts) for e in elements
    ) / len(elements)

    print(numerator / denominator)
    lst_values.append(numerator / denominator)
print(lst_values)

In [ ]:
# Post-process by post-selection.
post_selection_values = []

for i in range(len(nvals)):
    num_zero_zero = all_counts[i].get("0" * nvals[i] + "0" * nvals[i], 0)
    num_zero_one  = all_counts[i].get("0" * nvals[i] + "1" * nvals[i], 0)
    num_one_zero  = all_counts[i].get("1" * nvals[i] + "0" * nvals[i], 0)
    num_one_one   = all_counts[i].get("1" * nvals[i] + "1" * nvals[i], 0)

    total = num_zero_zero + num_zero_one + num_one_zero + num_one_one

    eval = (num_zero_zero - num_zero_one - num_one_zero + num_one_one) / total
    post_selection_values.append(eval)

print(post_selection_values)
lst_values = post_selection_values

In [ ]:
for i in range(len(nvals)):
    nzeros = all_counts[i].get("0" * nvals[i] * k)
    nones  =  all_counts[i].get("1" * nvals[i] * k)

    print((nzeros - nones) / (nzeros + nones))

## Plot results

In [ ]:
import numpy as np

xvals = [1] + nvals
yvals = [raw_value] + lst_values

save_key = f"{computer.name}_logical_bell_n357_depth_{depth}_{time_key}_job_id_{job.job_id()}_batch_job_id_{job_batch.job_id()}"

plt.plot(xvals, yvals, "--o", ms=10, mec="black", alpha=0.88)

plt.xlabel("$n$")
plt.ylabel(r"$\langle \bar{Z} \bar{Z} \rangle$");
plt.title(f"{computer.name}\nJob ids {job.job_id()} & {job_batch.job_id()}")
# plt.savefig(f"{save_key}.pdf")

In [ ]:
for n, counts in zip(nvals, all_counts):
    print(n, counts.get("0" * n * k))

In [ ]:
opcounts = []
for nqubits, circuit in zip(xvals, [compiled_raw] + to_run):
    print(circuit.count_ops())
    oneq = 0
    twoq = 0
    for gate, count in list(circuit.count_ops().items()):
        if gate in ("rz", "sx", "x"):
            oneq += count
        if gate in ("ecr", "cz", "cx"):
            twoq += count
    print("Oneq:", oneq)
    print("Twoq:", twoq)
    print("Total:", oneq + twoq)
    opcounts.append([nqubits, oneq, twoq])

## Save all data

In [20]:
# Save the opcounts.
np.savetxt(f"{save_key}_counts.txt", opcounts, header="nqubits, one-qubit gates, two-qubit gates")

In [21]:
# Save the nqubits, expectation values data.
data = np.array([xvals, yvals]).T

np.savetxt(f"{save_key}_expectation_values.txt", data)

In [22]:
# Save the qubits used.
qubits = [list(x) for x in layout.values()]

import pandas as pd

df = pd.DataFrame(qubits)
df.to_csv(f"{save_key}_qubits.txt", header=False)